In [ ]:
%autosave 120
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
#import spacy
#pip install nltk
from nltk.tokenize import word_tokenize
import numpy as np
#import en_core_web_sm
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import mutual_info_classif, f_classif
from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from pprint import pprint
from time import time
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from collections import defaultdict

# Load Data

In [ ]:
comments = pd.read_csv("data/comments_preprocessed_final.csv")
comments_50k = comments.iloc[:50000,]


# Naive Bayes

In [ ]:
X_50k = comments_50k.comment_text
y_50k = comments_50k.target
X = comments.comment_text
y = comments.target


In [ ]:
percent = 0.8
train_size = int(X.shape[0] * percent)
X_train = X.iloc[:train_size,]
X_valid = X.iloc[train_size:,]
y_train = y.iloc[:train_size,]
y_valid = y.iloc[train_size:,]


In [ ]:
train_size

In [ ]:
# use count vectorizer
# partly retreived from https://www.kaggle.com/catris25/multinomial-naive-bayes-with-countvectorizer

vectorizer = CountVectorizer()

X_train_cv = vectorizer.fit_transform(X_train)
X_valid_cv = vectorizer.transform(X_valid)

mnb = MultinomialNB()
mnb.fit(X_train_cv, y_train)
y_pred_nb = mnb.predict(X_valid_cv)

print("Accuracy:",metrics.accuracy_score(y_valid, y_pred_nb))


In [ ]:
# output nb with cv csv
y_pred_nb_df = pd.DataFrame({'pred':y_pred_nb, 'id': list(y_valid.index)})
y_result = y_pred_nb_df.merge(y_valid, left_on='id', right_on=y_valid.index).set_index('id')
y_result.to_csv('result_nb_8-2.csv')

In [ ]:
print('------------------------------')
print('Result for Multinomial Naive Bayes')
accuracy = round(accuracy_score(y_valid, y_pred_nb), 4)
print('Accuracy: %.4f' % accuracy)
precision = round(precision_score(y_valid, y_pred_nb), 4)
print('Precision: %.4f' % precision)
recall = round(recall_score(y_valid, y_pred_nb), 4)
print('Recall: %.4f' % recall)
score = round(f1_score(y_valid, y_pred_nb), 4)
print('F1_score: %.4f' % score)
print('------------------------------')

# Plain SVM

In [ ]:
percent = 0.8
train_size = int(X.shape[0] * percent)
X_train = X.iloc[:train_size,]
X_valid = X.iloc[train_size:,]
y_train = y.iloc[:train_size,]
y_valid = y.iloc[train_size:,]


In [ ]:
# Grid Search
pipeline = Pipeline([
                    ('tfidf', TfidfVectorizer()), 
                    ('SVC', svm.SVC()), 
                    ])
parameters = {
    'tfidf__stop_words': ['english'],
    'tfidf__max_features': [1000, 5000, 10000, 50000],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'SVC__C': [1e-2, 1e-1, 1e0, 1e1, 1e2],
    'SVC__kernel': ['linear'],
    'SVC__gamma': ['auto'],
    'SVC__max_iter': [5e2, 1e3, 5e3]
}

if __name__ == '__main__':
    grid_search=GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10)
    
    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)
    t0 = time()
    
    grid_search.fit(X, y)
    
    print("done in %0.3fmin" % ((time() - t0)/60))
    print()
    
    results = grid_search.cv_results_
    test_scores = results['mean_test_score']
    params = results['params']
    rank = results['rank_test_score']
    
    print("best score: %0.4f" % grid_search.best_score_)
    best_parameters = grid_search.best_estimator_.get_params()
    print("Best parameters set: ")
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

    #print("Top 5 best parameter sets based on accuracy score: ")
    #list(np.take(params,list(rank.argsort()[:5])))
    
    


In [ ]:
# use optimal parameters for SVM
tf_vectorizer_svm = TfidfVectorizer(max_features=50000, ngram_range = (1, 2), stop_words = 'english')
tf_vectorizer_svm.fit(X_train)
X_train_svm = tf_vectorizer_svm.transform(X_train)
X_valid_svm = tf_vectorizer_svm.transform(X_valid)

SVM = svm.SVC(C=10, kernel='linear', gamma='auto', max_iter = 1000)
SVM.fit(X_train_svm,y_train)
y_pred_svm = SVM.predict(X_valid_svm)
print("SVM Accuracy Score -> ",accuracy_score(y_valid, y_pred_svm)*100)


In [ ]:
print('------------------------------')
print('Result for Support Vector Machine')
accuracy = accuracy_score(y_valid, y_pred_svm)
print('Accuracy: %.4f' % accuracy)
precision = precision_score(y_valid, y_pred_svm)
print('Precision: %.4f' % precision)
recall = recall_score(y_valid, y_pred_svm)
print('Recall: %.4f' % recall)
score = f1_score(y_valid, y_pred_svm)
print('F1_score: %.4f' % score)
print('------------------------------')

In [ ]:
# output csv
y_pred_svm_df = pd.DataFrame({'pred':y_pred_svm, 'id': list(y_valid.index)})
y_result = y_pred_svm_df.merge(y_valid, left_on='id', right_on=y_valid.index).set_index('id')
y_result.to_csv('result_svm_8-2.csv')